<a href="https://colab.research.google.com/github/Nehaasati/DSA_1/blob/master/cats_nd_dog_classifiction_using_INCEPTION_MODEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:


import keras
from tensorflow.keras.layers import Layer
import tensorflow.keras.backend as K
import tensorflow as tf
from tensorflow.keras.datasets import cifar10

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPool2D,  \
    Dropout, Dense, Input, concatenate,      \
    GlobalAveragePooling2D, AveragePooling2D,\
    Flatten
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.utils.data_utils import Sequence


import os

from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Model 
import numpy as np
import matplotlib.pyplot as plt
import cv2 
import numpy as np 
from tensorflow.keras.datasets import cifar10 
from tensorflow.keras import backend as K 


Using TensorFlow backend.


In [0]:
_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'

path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=_URL, extract=True)

PATH = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')
IMAGE_WIDTH = 300
IMAGE_HEIGHT = 300
BATCH_SIZE =32
epochs =15

In [0]:
train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')

In [0]:
print(train_dir)
print(validation_dir)

/root/.keras/datasets/cats_and_dogs_filtered/train
/root/.keras/datasets/cats_and_dogs_filtered/validation


In [0]:
train_cats_dir = os.path.join(train_dir, 'cats')  # directory with our training cat pictures
train_dogs_dir = os.path.join(train_dir, 'dogs')  # directory with our training dog pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')  # directory with our validation cat pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

In [0]:
num_cats_tr = len(os.listdir(train_cats_dir))
num_dogs_tr = len(os.listdir(train_dogs_dir))

num_cats_val = len(os.listdir(validation_cats_dir))
num_dogs_val = len(os.listdir(validation_dogs_dir))

total_train = num_cats_tr + num_dogs_tr
total_val = num_cats_val + num_dogs_val
print('total training cat images:', num_cats_tr)
print('total training dog images:', num_dogs_tr)

print('total validation cat images:', num_cats_val)
print('total validation dog images:', num_dogs_val)
print("--")
print("Total training images:", total_train)
print("Total validation images:", total_val)

total training cat images: 1000
total training dog images: 1000
total validation cat images: 500
total validation dog images: 500
--
Total training images: 2000
Total validation images: 1000


In [0]:
 image_gen_train = ImageDataGenerator(
                    
                    rotation_range=45,
                    width_shift_range=.15,
                    height_shift_range=.15,
                    horizontal_flip=True,
                    zoom_range=0.5
                    )

In [0]:
validation_image_generator = ImageDataGenerator(rescale=1./255)

In [0]:
training_data_generator = image_gen_train.flow_from_directory(train_dir, 
                                        target_size=(224, 224),
                                        color_mode='rgb',
                                        #classes=None,
                                        class_mode='binary',
                                        batch_size=32, 
                                        shuffle=True) #keep data ordered 

num_samples = len(training_data_generator.filenames)
num_classes = len(training_data_generator.class_indices)




Found 2000 images belonging to 2 classes.


In [0]:
val_data_gen = validation_image_generator.flow_from_directory(validation_dir, 
                                        target_size=(300, 300),
                                        color_mode='rgb',
                                        #classes=None,
                                        class_mode='binary',
                                        batch_size=32, 
                                        shuffle=True) #keep data ordered 

num_samples = len(val_data_gen.filenames)
num_classes = len(val_data_gen.class_indices)


Found 1000 images belonging to 2 classes.


In [0]:
val_data_gen = validation_image_generator.flow_from_directory(batch_size=BATCH_SIZE,
                                                              directory=validation_dir,
                                                              target_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
                                                              class_mode='binary')

    

Found 1000 images belonging to 2 classes.


In [0]:
def inception_module(x,
                     filters_1x1,
                     filters_3x3_reduce,
                     filters_3x3,
                     filters_5x5_reduce,
                     filters_5x5,
                     filters_pool_proj,
                     name=None):
    
    conv_1x1 = Conv2D(filters_1x1, (1, 1), padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform())(x)
    
    conv_3x3 = Conv2D(filters_3x3_reduce, (1, 1), padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform())(x)
    conv_3x3 = Conv2D(filters_3x3, (3, 3), padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform())(conv_3x3)

    conv_5x5 = Conv2D(filters_5x5_reduce, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    conv_5x5 = Conv2D(filters_5x5, (5, 5), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_5x5)

    pool_proj = MaxPool2D((3, 3), strides=(1, 1), padding='same')(x)
    pool_proj = Conv2D(filters_pool_proj, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(pool_proj)

    output = concatenate([conv_1x1, conv_3x3, conv_5x5, pool_proj], axis=3, name=name)
    
    return output

In [0]:
#Input=training_data_generator 

In [0]:
input= Input(shape=(224, 224, 3))
x = Conv2D(64, (7, 7), padding='same', strides=(2, 2), activation='relu', kernel_initializer=keras.initializers.glorot_uniform())(input)
x = MaxPool2D((3, 3), padding='same', strides=(2, 2))(x)
x = Conv2D(64, (1, 1), padding='same', strides=(1, 1), activation='relu')(x)
x = Conv2D(192, (3, 3), padding='same', strides=(1, 1), activation='relu')(x)
x = MaxPool2D((3, 3), padding='same', strides=(2, 2))(x)

#first  inception_module 
conv_1x1 = Conv2D(64, (1, 1), padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform())(input)
conv_3x3 = Conv2D(96, (1, 1), padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform())(input)
conv_3x3 = Conv2D(128, (3, 3), padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform())(conv_3x3)

conv_5x5 = Conv2D(16, (1, 1), padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform())(input)
conv_5x5 = Conv2D(32, (5, 5), padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform())(conv_5x5)

pool_proj = MaxPool2D((3, 3), strides=(1, 1), padding='same')(input)
pool_proj = Conv2D(32, (1, 1), padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform())(pool_proj)
x = concatenate([conv_1x1, conv_3x3, conv_5x5, pool_proj], axis=3) 

#second insception
conv_1x1 = Conv2D(128, (1, 1), padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform())(input)
conv_3x3 = Conv2D(128, (1, 1), padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform())(input)
conv_3x3 = Conv2D(192, (3, 3), padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform())(conv_3x3)

conv_5x5 = Conv2D(32, (1, 1), padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform())(input)
conv_5x5 = Conv2D(96, (5, 5), padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform())(conv_5x5)

pool_proj = MaxPool2D((3, 3), strides=(1, 1), padding='same')(input)
pool_proj = Conv2D(64, (1, 1), padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform())(pool_proj)
x = concatenate([conv_1x1, conv_3x3, conv_5x5, pool_proj], axis=3) 

####max pool

x = MaxPool2D((3, 3), padding='same', strides=(2, 2))(x)

#4a insception
conv_1x1 = Conv2D(192, (1, 1), padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform())(input)
conv_3x3 = Conv2D(96, (1, 1), padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform())(input)
conv_3x3 = Conv2D(208, (3, 3), padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform())(conv_3x3)

conv_5x5 = Conv2D(16, (1, 1), padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform())(input)
conv_5x5 = Conv2D(48, (5, 5), padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform())(conv_5x5)

pool_proj = MaxPool2D((3, 3), strides=(1, 1), padding='same')(input)
pool_proj = Conv2D(64, (1, 1), padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform())(pool_proj)
x = concatenate([conv_1x1, conv_3x3, conv_5x5, pool_proj], axis=3) 

##avg pool

x1 = AveragePooling2D((5, 5), strides=3)(x)
x1 = Conv2D(128, (1, 1), padding='same', activation='relu')(x1)
x1 = Flatten()(x1)
#x1 = Dense(1024, activation='relu')(x1)
x1 = Dense(2, activation='relu')(x1)
x1 = Dropout(0.7)(x1)
x1 = Dense(2, activation='softmax')(x1)

#4b insception

conv_1x1 = Conv2D(160, (1, 1), padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform())(input)
conv_3x3 = Conv2D(112, (1, 1), padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform())(input)
conv_3x3 = Conv2D(224, (3, 3), padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform())(conv_3x3)

conv_5x5 = Conv2D(24, (1, 1), padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform())(input)
conv_5x5 = Conv2D(64, (5, 5), padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform())(conv_5x5)

pool_proj = MaxPool2D((3, 3), strides=(1, 1), padding='same')(input)
pool_proj = Conv2D(64, (1, 1), padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform())(pool_proj)
x = concatenate([conv_1x1, conv_3x3, conv_5x5, pool_proj], axis=3) 

##4c insception
conv_1x1 = Conv2D(128, (1, 1), padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform())(input)
conv_3x3 = Conv2D(128, (1, 1), padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform())(input)
conv_3x3 = Conv2D(256, (3, 3), padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform())(conv_3x3)

conv_5x5 = Conv2D(24, (1, 1), padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform())(input)
conv_5x5 = Conv2D(64, (5, 5), padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform())(conv_5x5)

pool_proj = MaxPool2D((3, 3), strides=(1, 1), padding='same')(input)
pool_proj = Conv2D(64, (1, 1), padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform())(pool_proj)
x = concatenate([conv_1x1, conv_3x3, conv_5x5, pool_proj], axis=3) 

##4d insception
conv_1x1 = Conv2D(112, (1, 1), padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform())(input)
conv_3x3 = Conv2D(144, (1, 1), padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform())(input)
conv_3x3 = Conv2D(288, (3, 3), padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform())(conv_3x3)

conv_5x5 = Conv2D(32, (1, 1), padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform())(input)
conv_5x5 = Conv2D(64, (5, 5), padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform())(conv_5x5)

pool_proj = MaxPool2D((3, 3), strides=(1, 1), padding='same')(input)
pool_proj = Conv2D(64, (1, 1), padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform())(pool_proj)
x = concatenate([conv_1x1, conv_3x3, conv_5x5, pool_proj], axis=3) 

###x2 avg pool
x2 = AveragePooling2D((5, 5), strides=3)(x)
x2 = Conv2D(128, (1, 1), padding='same', activation='relu')(x2)
x2 = Flatten()(x2)
#x2 = Dense(1024, activation='relu')(x2)
x2 = Dense(2, activation='relu')(x2)
x2 = Dropout(0.7)(x2)
x2 = Dense(2, activation='softmax')(x2)

##4e insception
conv_1x1 = Conv2D(256, (1, 1), padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform())(input)
conv_3x3 = Conv2D(160, (1, 1), padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform())(input)
conv_3x3 = Conv2D(320, (3, 3), padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform())(conv_3x3)

conv_5x5 = Conv2D(32, (1, 1), padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform())(input)
conv_5x5 = Conv2D(128, (5, 5), padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform())(conv_5x5)

pool_proj = MaxPool2D((3, 3), strides=(1, 1), padding='same')(input)
pool_proj = Conv2D(128, (1, 1), padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform())(pool_proj)
x = concatenate([conv_1x1, conv_3x3, conv_5x5, pool_proj], axis=3)

##max poll
x = MaxPool2D((3, 3), padding='same', strides=(2, 2))(x)

##5a insception
conv_1x1 = Conv2D(256, (1, 1), padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform())(input)
conv_3x3 = Conv2D(160, (1, 1), padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform())(input)
conv_3x3 = Conv2D(320, (3, 3), padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform())(conv_3x3)

conv_5x5 = Conv2D(32, (1, 1), padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform())(input)
conv_5x5 = Conv2D(128, (5, 5), padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform())(conv_5x5)

pool_proj = MaxPool2D((3, 3), strides=(1, 1), padding='same')(input)
pool_proj = Conv2D(128, (1, 1), padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform())(pool_proj)
x = concatenate([conv_1x1, conv_3x3, conv_5x5, pool_proj], axis=3)

##5b insception
conv_1x1 = Conv2D(384, (1, 1), padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform())(input)
conv_3x3 = Conv2D(192, (1, 1), padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform())(input)
conv_3x3 = Conv2D(384, (3, 3), padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform())(conv_3x3)

conv_5x5 = Conv2D(48, (1, 1), padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform())(input)
conv_5x5 = Conv2D(128, (5, 5), padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform())(conv_5x5)

pool_proj = MaxPool2D((3, 3), strides=(1, 1), padding='same')(input)
pool_proj = Conv2D(128, (1, 1), padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform())(pool_proj)
x = concatenate([conv_1x1, conv_3x3, conv_5x5, pool_proj], axis=3) 
x = GlobalAveragePooling2D()(x)

x = Dropout(0.4)(x)

x = Dense(2, activation='softmax',kernel_initializer =keras.initializers.glorot_uniform())(x)

model = Model(input, [x, x1, x2])


In [0]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_81 (Conv2D)              (None, 224, 224, 96) 384         input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_83 (Conv2D)              (None, 224, 224, 16) 64          input_3[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_19 (MaxPooling2D) (None, 224, 224, 3)  0           input_3[0][0]                    
____________________________________________________________________________________________